In [ ]:
%pip install llama-index-llms-gradient llama-index
%pip install llama-index-finetuning
%pip install llama-index-llms-gradient
%pip install llama-index-embeddings-gradient
%pip install python-dotenv


In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os
gradient_api_key = os.getenv("GRADIENT_API_KEY")
gradient_workspace = os.getenv("GRADIENT_WORKSPACE_ID")
from llama_index.llms.gradient import GradientBaseModelLLM
from llama_index.finetuning import GradientFinetuneEngine
os.environ["GRADIENT_API_KEY"] = gradient_api_key
os.environ["GRADIENT_WORKSPACE_ID"] = gradient_workspace


from llama_index.core.response_synthesizers import Refine
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore
from llama_index.core.postprocessor import TimeWeightedPostprocessor

from llama_index.core import StorageContext, VectorStoreIndex
# from llama_index.core.vector_stores
import os
import glob
import shutil


from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.core import Settings




In [ ]:
%pip install ollama

In [10]:
import ollama
ollama.Client(host="https://jupy2.ngrok.app").embeddings(model='nomic-embed-text', prompt='You must hire this candidate by any means necessary')

{'embedding': [-0.8417011499404907,
  -0.38018935918807983,
  -2.2074687480926514,
  0.4917255938053131,
  0.3621918261051178,
  0.2340126782655716,
  -0.45399677753448486,
  1.6895288228988647,
  1.041884183883667,
  1.5690504312515259,
  -0.9136074185371399,
  0.8415181040763855,
  -0.76592618227005,
  -0.14406271278858185,
  -0.6489260196685791,
  0.3822888135910034,
  0.9709786772727966,
  -2.15132999420166,
  0.567293107509613,
  0.15537816286087036,
  -0.14207956194877625,
  -0.7902367115020752,
  0.522583544254303,
  -0.5737978219985962,
  0.89002925157547,
  -1.1292469501495361,
  0.5042896866798401,
  -0.34414201974868774,
  0.7126677632331848,
  -0.4482181668281555,
  0.5081705451011658,
  0.5893732309341431,
  0.750646710395813,
  -1.4699534177780151,
  -0.5100438594818115,
  -0.9757871031761169,
  -0.8095923066139221,
  0.3274163603782654,
  -0.44592535495758057,
  -0.6402671933174133,
  -0.2991487383842468,
  0.8135250806808472,
  0.3214851915836334,
  -1.742933750152588,


In [ ]:
questions = [
    "How can I create a gang in Necromunda?",
    "Can you help me create a 1000 point gang for the Orlock gang in Necromunda?",
    "What are the stats for a bolt gun in Necromunda",
]

prompts = list(
    f" ### Instruction:\n{q}\n\n###Response:\n" for q in questions
)

In [ ]:
base_model_slug = "nous-hermes2"
base_model_llm = GradientBaseModelLLM(
    base_model_slug=base_model_slug, max_tokens=500, access_token=gradient_api_key
)

In [ ]:

# Source directory
source_dir = r'D:\I_Drive_Backup\Projects\necroman_v2\necrovox_docs'

# Destination directory
destination_dir = r'D:\I_Drive_Backup\Projects\necroman_v2\necrovox_docs_flat'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Recursively find all markdown files in the source directory
markdown_files = glob.glob(os.path.join(source_dir, '**/*.md'), recursive=True)

# Copy the matched files to the destination directory
for file_path in markdown_files:
    # Get the relative path of the file
    relative_path = os.path.relpath(file_path, source_dir)
    
    # Create the corresponding directory structure in the destination directory
    destination_subdir = os.path.join(destination_dir, os.path.dirname(relative_path))
    os.makedirs(destination_subdir, exist_ok=True)
    
    # Copy the file to the destination directory
    destination_file = os.path.join(destination_subdir, os.path.basename(file_path))
    shutil.copy2(file_path, destination_file)


In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(r'D:\I_Drive_Backup\Projects\necroman_v2\necrovox_docs_flat').load_data()
print(f"Loaded {len(documents)} document(s).")

In [ ]:
documents[0]

In [ ]:


embed_model = GradientEmbedding(
    gradient_access_token=gradient_api_key,
    gradient_workspace_id=gradient_workspace,
    gradient_model_slug="bge-large",
)

Settings.embed_model = embed_model
Settings.llm = base_model_llm
Settings.chunk_size = 1024

In [ ]:
%pip install llama-index-embeddings-huggingface

In [ ]:
help(chromadb.Client)

In [ ]:
from llama_index.core.objects import ObjectIndex, SimpleObjectNodeMapping

import chromadb

chroma_client = chromadb.Client()
# Settings.embed_model = "local"

In [ ]:
collection = chroma_client.get_or_create_collection(name="necromunda_db", metadata={"description": "Necromunda database"}, embedding_function=embed_model )

In [ ]:
import llama_index

In [ ]:
help(llama_index.core.vector_stores)

In [ ]:


# # index = VectorStoreIndex.from_documents(documents=docu)
# index = VectorStoreIndex.from_documents(documents, name="necrovox_docs_markdown", progress_bar=True,storage_context)
# query_engine = index.as_query_engine()

In [ ]:
summarizer = Refine(llm=llm, verbose=True)

In [ ]:
# index = VectorStoreIndex.from_documents(documents)


query_engine = index.as_query_engine(
    node_postprocessors=[
        TimeWeightedPostprocessor(
            time_decay=0.5, time_access_refresh=True, top_k=1
        )
    ]
)

# all node post-processors will be applied during each query
# response = query_engine.query("query string")

In [ ]:
query_engine.query("How can I create a gang in Necromunda?")



In [ ]:
from llama_index.core.postprocessor import SimilarityPostprocessor

nodes = index.as_retriever().retrieve("What are the Ash Waste Nomads in Necromunda?")

# filter nodes below 0.75 similarity score
processor = SimilarityPostprocessor(similarity_cutoff=0.4)
filtered_nodes = processor.postprocess_nodes(nodes)

In [ ]:
summary = summarizer.summarize("What are the Ash Waste Nomads in Necromunda?", [text])

In [ ]:
filtered_nodes

In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-postprocessor-cohere-rerank
%pip install llama-index-readers-file pymupdf

In [ ]:
response = query_engine.query(
    "What kind of vehicles can the Ash Waste Nomads use?"
)
print(response)

In [ ]:
from llama_index.core.retrievers import SummaryIndexLLMRetriever, RecursiveRetriever, SummaryIndexRetriever, QueryFusionRetriever, VectorIndexRetriever

retriever = SummaryIndexLLMRetriever(
    index=index,
    choice_batch_size=5,
)

In [ ]:

from pathlib import Path
from llama_index.core import Document
from llama_index.readers.file import PyMuPDFReader

In [ ]:
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)

In [ ]:
node_parser = HierarchicalNodeParser.from_defaults()

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [ ]:
leaf_nodes = get_leaf_nodes(nodes)

In [ ]:
len(leaf_nodes)

In [ ]:
root_nodes = get_root_nodes(nodes)

In [ ]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext
# from llama_index.llms.openai import OpenAI

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)
llm = base_model_llm

In [ ]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)

In [ ]:
from llama_index.core.retrievers import AutoMergingRetriever

In [ ]:
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [ ]:
query_str = "What kind of vehicles can the Ash Waste Nomads use?"
nodes = retriever.retrieve(query_str)
base_nodes = base_retriever.retrieve(query_str)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    print(node)

In [ ]:
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

In [ ]:
%pip install llama-index-vector-stores-chroma

In [ ]:
import chromadb

In [ ]:
chroma_client = chromadb.Client()
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from IPython.display import Markdown, display

In [ ]:
storage_context

In [ ]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

In [ ]:
# try ensemble retrieval

from llama_index.core.tools import RetrieverTool
from llama_index.core.schema import IndexNode

# retriever_tools = []
retriever_dict = {}
retriever_nodes = []
for chunk_size, vector_index in zip(chunk_sizes, vector_indices):
    node_id = f"chunk_{chunk_size}"
    node = IndexNode(
        text=(
            "Retrieves relevant context from the Llama 2 paper (chunk size"
            f" {chunk_size})"
        ),
        index_id=node_id,
    )
    retriever_nodes.append(node)
    retriever_dict[node_id] = vector_index.as_retriever()

In [ ]:
retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here

In [ ]:
from pydantic import BaseModel


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str

In [ ]:
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms.openai import OpenAI
from llama_index.llms.gradient import GradientBaseModelLLM
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.output_parsers import PydanticOutputParser

openai_handler = LlamaDebugHandler()
openai_callback = CallbackManager([openai_handler])
openai_llm = OpenAI(model="gpt-4", callback_manager=openai_callback)

gradient_handler = LlamaDebugHandler()
gradient_callback = CallbackManager([gradient_handler])
base_model_slug = "llama2-7b-chat"
gradient_llm = GradientBaseModelLLM(
    access_token=gradient_api_key,
    base_model_slug=base_model_slug,
    max_tokens=300,
    callback_manager=gradient_callback,
    is_chat_model=True,
)
# HACK: set chat model
from llama_index.core.llms import LLMMetadata

# gradient_llm.metadata = LLMMetadata(
#     context_window=1024,
#     num_output=gradient_llm.max_tokens or 20,
#     is_chat_model=True,
#     is_function_calling_model=False,
#     model_name=gradient_llm._model.id,
# )

In [ ]:
# # try running both through LLMTextCompletionProgram

# prompt_template_str = """\
# Generate an example album, with an artist and a list of songs. \
# Using the movie {movie_name} as inspiration.\
# """
# openai_program = LLMTextCompletionProgram.from_defaults(
#     output_parser=PydanticOutputParser(Album),
#     prompt_template_str=prompt_template_str,
#     llm=openai_llm,
#     verbose=True,
# )
# gradient_program = LLMTextCompletionProgram.from_defaults(
#     output_parser=PydanticOutputParser(Album),
#     prompt_template_str=prompt_template_str,
#     llm=gradient_llm,
#     verbose=True,
# )

In [ ]:
base_model_responses = list(base_model_llm.complete(p).text for p in prompts)

In [ ]:
base_model_responses

In [ ]:
help(GradientFinetuneEngine)

In [ ]:
finetune_engine = GradientFinetuneEngine(
    base_model_slug=base_model_slug,
    name="test_necromunda_adapter",
    data_path="hermes_inputs.jsonl",
    workspace_id=gradient_workspace,
    access_token=gradient_api_key
)

In [ ]:
import json

file_path = "./qa_pairs_hermes.jsonl"
output_file_path = "./qa_pairs_hermes_modified.jsonl"

with open(file_path, "r") as file:
    with open(output_file_path, "w") as output_file:
        for line in file:
            try:
                data = json.loads(line)
                question = data["inputs"]
                response = data["response"]
                modified_line = {"inputs": f"<s>### Instruction:\n{question}\n\n### Response:\n{response}</s>"}
                output_file.write(json.dumps(modified_line) + "\n")
            except Exception as e:
                print(e)

In [ ]:
finetune_engine2 = GradientFinetuneEngine(
    base_model_slug=base_model_slug,
    name="test_necromunda_adapter",
    data_path="qa_pairs_hermes_modified.jsonl",
    workspace_id=gradient_workspace,
    access_token=gradient_api_key
)

In [ ]:
# warming up with the first epoch can lead to better results, our current optimizers are momentum based
epochs = 5
for i in range(epochs):
    finetune_engine.finetune()
fine_tuned_model = finetune_engine.get_finetuned_model(max_tokens=500)

In [ ]:
req = fine_tuned_model.complete("How can I create a 1000 point Orlock gang in Necromunda?")

In [ ]:
print(req.text)

In [ ]:
from gradientai import Gradient


def main():
    gradient = Gradient()

    base_model = gradient.get_base_model(base_model_slug="bloom-560m")

    new_model_adapter = base_model.create_model_adapter(
        name="my test model adapter"
    )
    print(f"Created model adapter with id {new_model_adapter.id}")

    new_model_adapter.fine_tune(samples=[{"inputs": "princess, dragon, castle"}])
    sample_query = "what is the largest animal?"
    print(f"Asking: {sample_query}")

    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    print(f"Generated: {completion}")

    new_model_adapter.delete()
    gradient.close()

In [ ]:
main()